# Baseline Model

Here, we're going to establish a  baseline model for our audio classification model. The baseline will use, as defined in the "models.py" class, will be a 2 layer neural network, with a hidden layer size of 20. 

There is a dropout layer with a 0.75 just after the hidden layer and before the output softmax layer. 

We use a standard scaler because our features are on different scales (look at data prep for more info), which will ensure we get more effecient/effective error minimization.

Afterwards, we will expand this baseline by adding more hidden layers and the necessary model regularization.

We're going to see how this baseline performs with 3 different feature selection methods, so we don't have to try out these permutations later (it's difficult to integrate all these different feature selection methods with sklearn random search). So, we will simply choose the feature selection method that gives the best validation results on the baseline model.

In [1]:
# so we have access to the Google Drive filesystem
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# necessary imports
import os
import pandas as pd
import numpy as np

# so we can access local modules within Colab
os.chdir('/content/drive/My Drive/auto-age-detector-model')

# feature selection defined functions
from feature_selection import lasso_feature_selection
from feature_selection import tree_based_feature_selection
from feature_selection import chi_squared_feature_selection
from feature_selection import pca_feature_selection

# baseline model creation
from models import baseline_model

# for feature scaling
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


Here, we import the training data, omitting feature unneccessary for our model. Then we split it into our inputs and outputs.

In [0]:
df_train = pd.read_csv('data/audio_training_data_cleaned.csv').drop(columns=['Unnamed: 0','filename'])
# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)
X_train = df_train.drop(columns=['age'],axis=1)
y_train = df_train['age']

We one hot encode the outputs so we can build a multiclass classification model.

In [0]:
replaced = {'teens':0,'twenties':1,'thirties':2,'fourties':3,'fifties':4,
            'sixties':5,'seventies':6,'eighties':7}

# https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array

# need to put one hot encoded in keras model
y_train_ohe = y_train.replace(replaced)
y_train_ohe = np.eye(np.max(y_train_ohe)+1)[y_train_ohe]

Now we're going to fit a model with different feature selection methods.

We're going to use a validation split of 0.2, 15 epochs, and a batch size of 32 for these initial baselines. 

## Baseline with L1 feature selection - Logistic Regression

In [5]:
# feature scaling before to help with convergence
scaler = StandardScaler()
scaler.fit(X_train)
X_train_l1_log_reg = scaler.transform(X_train)

X_train_l1_log_reg,data_transformer = lasso_feature_selection(X_train_l1_log_reg,
                                                              y_train,
                                                              model='logreg')

print(f'Reduced to {X_train_l1_log_reg.shape[1]} features.')


model_l1_log_reg = baseline_model()
model_l1_log_reg.fit(X_train_l1_log_reg,y_train_ohe,batch_size=32,
                    validation_split=0.2,epochs=10)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Reduced to 59 features.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 59012 samples, validate on 14753 samples
Epoch 1/10





59012/59012 [==============================] - 11s 178us/step - loss: 1.7268 - acc: 0.3344 - val_loss: 1.5470 - val_acc: 0.3954
Epoch 2/10
59012/59012 [==============================] - 9s 160us/step - loss: 1.5583 - acc: 0.3863 - val_loss: 1.5033 - val_acc: 0.4146
Epoch 3/10
59012/59012 [==============================] - 9s 158us/step - loss: 1.5346 - acc: 0.3989 - val_loss: 1.4827 - val_acc: 0.4241
Epoch 4/10
59012/59012 [==============================] - 10s 166us/step - loss: 1.5154 - acc: 0.4077 - val_loss: 1.4631 - val_acc: 0.4314
Epoch 5/10
59012/59012 [==============================] - 10s 165us/step - loss: 1.5030 - acc: 0.4111 - val_loss: 1.4523 - val_acc: 0.4345
Epoch 6/10

## Baseline with tree-based feature selection

In [6]:
X_train_tree,data_transformer = tree_based_feature_selection(X_train,y_train,
                                                        n_estimators=75)

print(f'Reduced to {X_train_tree.shape[1]} features.')
# feature scaling after
scaler = StandardScaler()
scaler.fit(X_train_tree)
X_train_tree = scaler.transform(X_train_tree)

model_tree = baseline_model()
model_tree.fit(X_train_tree,y_train_ohe,batch_size=32,
                    validation_split=0.15,epochs=10)

Reduced to 83 features.
Train on 62700 samples, validate on 11065 samples
Epoch 1/10
62700/62700 [==============================] - 10s 166us/step - loss: 1.6892 - acc: 0.3410 - val_loss: 1.5344 - val_acc: 0.4023
Epoch 2/10
62700/62700 [==============================] - 11s 171us/step - loss: 1.5409 - acc: 0.3919 - val_loss: 1.4823 - val_acc: 0.4215
Epoch 3/10
62700/62700 [==============================] - 10s 156us/step - loss: 1.5094 - acc: 0.4067 - val_loss: 1.4554 - val_acc: 0.4359
Epoch 4/10
62700/62700 [==============================] - 10s 156us/step - loss: 1.4899 - acc: 0.4152 - val_loss: 1.4404 - val_acc: 0.4456
Epoch 5/10
62700/62700 [==============================] - 10s 165us/step - loss: 1.4767 - acc: 0.4235 - val_loss: 1.4320 - val_acc: 0.4456
Epoch 6/10
62700/62700 [==============================] - 10s 165us/step - loss: 1.4670 - acc: 0.4263 - val_loss: 1.4207 - val_acc: 0.4497
Epoch 7/10
62700/62700 [==============================] - 10s 160us/step - loss: 1.4582 - ac

## Baseline with chi_squared_based_feature selection

Try with 80 best features first.

In [7]:
X_train_chi2_80,data_transformer = chi_squared_feature_selection(X_train,
                                                              y_train)

print(f'Reduced to {X_train_chi2_80.shape[1]} features.')
# feature scaling after
scaler = StandardScaler()
scaler.fit(X_train_chi2_80)
X_train_chi2_80 = scaler.transform(X_train_chi2_80)

model_chi2_80 = baseline_model()
model_chi2_80.fit(X_train_chi2_80,y_train_ohe,batch_size=32,
                  validation_split=0.15,epochs=10)

Reduced to 80 features.
Train on 62700 samples, validate on 11065 samples
Epoch 1/10
62700/62700 [==============================] - 11s 173us/step - loss: 1.7183 - acc: 0.3316 - val_loss: 1.5796 - val_acc: 0.3814
Epoch 2/10
62700/62700 [==============================] - 10s 163us/step - loss: 1.5795 - acc: 0.3755 - val_loss: 1.5453 - val_acc: 0.3949
Epoch 3/10
62700/62700 [==============================] - 10s 157us/step - loss: 1.5536 - acc: 0.3840 - val_loss: 1.5222 - val_acc: 0.4033
Epoch 4/10
62700/62700 [==============================] - 10s 160us/step - loss: 1.5393 - acc: 0.3903 - val_loss: 1.5033 - val_acc: 0.4121
Epoch 5/10
62700/62700 [==============================] - 10s 159us/step - loss: 1.5283 - acc: 0.3925 - val_loss: 1.4921 - val_acc: 0.4140
Epoch 6/10
62700/62700 [==============================] - 10s 163us/step - loss: 1.5209 - acc: 0.4003 - val_loss: 1.4850 - val_acc: 0.4165
Epoch 7/10
62700/62700 [==============================] - 11s 174us/step - loss: 1.5145 - ac

Now try with 50 features to see if performance improves.

In [8]:
X_train_chi2_50,data_transformer = chi_squared_feature_selection(X_train,
                                                              y_train,k=50)

print(f'Reduced to {X_train_chi2_50.shape[1]} features.')
# feature scaling after
scaler = StandardScaler()
scaler.fit(X_train_chi2_50)
X_train_chi2_50 = scaler.transform(X_train_chi2_50)

model_chi2_50 = baseline_model()
model_chi2_50.fit(X_train_chi2_50,y_train_ohe,batch_size=32,
                  validation_split=0.15,epochs=10)

Reduced to 50 features.
Train on 62700 samples, validate on 11065 samples
Epoch 1/10
62700/62700 [==============================] - 10s 167us/step - loss: 1.7356 - acc: 0.3275 - val_loss: 1.6253 - val_acc: 0.3625
Epoch 2/10
62700/62700 [==============================] - 10s 158us/step - loss: 1.6203 - acc: 0.3618 - val_loss: 1.5949 - val_acc: 0.3784
Epoch 3/10
62700/62700 [==============================] - 10s 159us/step - loss: 1.5994 - acc: 0.3703 - val_loss: 1.5771 - val_acc: 0.3856
Epoch 4/10
62700/62700 [==============================] - 10s 157us/step - loss: 1.5898 - acc: 0.3729 - val_loss: 1.5630 - val_acc: 0.3879
Epoch 5/10
62700/62700 [==============================] - 10s 162us/step - loss: 1.5788 - acc: 0.3786 - val_loss: 1.5540 - val_acc: 0.3906
Epoch 6/10
62700/62700 [==============================] - 10s 156us/step - loss: 1.5712 - acc: 0.3802 - val_loss: 1.5472 - val_acc: 0.3965
Epoch 7/10
62700/62700 [==============================] - 10s 160us/step - loss: 1.5668 - ac

## PCA Dimensionality Reduction

In [11]:
# feature scaling because PCA is bias towards high magnitude features
scaler = StandardScaler()
scaler.fit(X_train)
X_train_pca = scaler.transform(X_train)

# get 80 best features
X_train_pca,data_transformer = pca_feature_selection(X_train_pca)

model_pca = baseline_model()
model_pca.fit(X_train_pca,y_train_ohe,batch_size=32,
                  validation_split=0.15,epochs=10)

Train on 62700 samples, validate on 11065 samples
Epoch 1/10
62700/62700 [==============================] - 11s 168us/step - loss: 1.6980 - acc: 0.3347 - val_loss: 1.5213 - val_acc: 0.3948
Epoch 2/10
62700/62700 [==============================] - 10s 164us/step - loss: 1.5342 - acc: 0.3924 - val_loss: 1.4679 - val_acc: 0.4216
Epoch 3/10
62700/62700 [==============================] - 10s 159us/step - loss: 1.5013 - acc: 0.4029 - val_loss: 1.4441 - val_acc: 0.4352
Epoch 4/10
62700/62700 [==============================] - 10s 165us/step - loss: 1.4866 - acc: 0.4104 - val_loss: 1.4245 - val_acc: 0.4444
Epoch 5/10
62700/62700 [==============================] - 10s 158us/step - loss: 1.4748 - acc: 0.4162 - val_loss: 1.4111 - val_acc: 0.4518
Epoch 6/10
62700/62700 [==============================] - 10s 161us/step - loss: 1.4656 - acc: 0.4210 - val_loss: 1.4045 - val_acc: 0.4572
Epoch 7/10
62700/62700 [==============================] - 11s 168us/step - loss: 1.4618 - acc: 0.4256 - val_loss: 1.

We've finished all feature selection methods for our baseline models. Luckily, all baseline models are decreasing training error after each epoch, which verifies that our neural network is fitting to the data.

Tree-based feature selection seemed to yield the best results, so we will use this type of feature selection going forward.